<a href="https://colab.research.google.com/github/thoadao0301/AI-Project/blob/NamTP/Set_up_output_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install keras_facenet

In [ ]:
import sys
import os, random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from tqdm import tqdm
from keras_facenet import FaceNet
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

# **Training classification**

---



## Prepare dataset

In [ ]:
# Gets a detection dict for each face
# in an image. Each one has the bounding box and
# face landmarks (from mtcnn.MTCNN) along with
# the embedding from FaceNet.
# return List of embedded images in folder
def get_embedding(dir, embedder):
    x = []
    for filename in listdir(dir):
        # set up file path
        filepath = os.path.join(dir,filename) 
        # load image from file
        image = Image.open(filepath)
        # convert to RGB, if needed
        image = image.convert('RGB')
        # convert to array
        pixels = np.asarray(image)
        # get embeddings for the faces in an image
        detections = embedder.extract(pixels, threshold=0.95)
        if(len(detections)<1):
            print('ERROR: Can\'t not detect face: ',filepath)
            continue
        if(len(detections)>1):
            print('ERROR: More than two face detected: ', filepath)
            continue
        x.append(detections[0]['embedding'])
    return x


In [ ]:
# Get embedded list images and list labels in folder
def load_dataset(dir, embedder):
    x , y = [],[]
    for subdir in listdir(dir):
        subdir_path = dir+'/'+subdir
        
        data_embedding = get_embedding(subdir_path, embedder)
        for data in data_embedding:
            x.append(data)
            y.append(subdir)
    return x, y

In [ ]:
# Create embedder tools with keras_facenet
embedder = FaceNet()

In [ ]:
# Create train dataset
dataset_train_path= '/content/drive/Shareddrives/AI_Project/DatasetVietNam/DataVietNam/VietNam/VN_Male' # Train data path
train_x, train_y = load_dataset(dataset_train_path, embedder)

In [ ]:
# Create test dataset
dataset_test_path= '/content/drive/Shareddrives/AI_Project/DatasetVietNam/DataVietNam/VietNam/VN_Male_Test' # Test data path
test_x, test_y = load_dataset(dataset_test_path, embedder)

## Training model classification using SVM method

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC

# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(train_y)
train_y = out_encoder.transform(train_y)
test_y = out_encoder.transform(test_y)

In [ ]:
# tranning model
model = SVC(kernel='linear', probability=True)
model.fit(train_x, train_y)
# score
score_train = model.score(train_x, train_y)
score_test = model.score(test_x, test_y)
# summarize
print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))

## Use model to recognize faces in images (test for show image and probrability)

In [1]:
# Get predict of image
import cv2
# read image from file path
image = cv2.imread('') # Image test path
# convert to RGB, if needed
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# convert to array
pixels = asarray(image)
# create the detector, using default weights
detections = embedder.extract(pixels)
# loop through each face in detections
for detect in detections:
    # get embedded image
    embedded_img = detect['embedding']

    # expand dimension for feeding data to the model
    samples = np.expand_dims(embedded_img, axis=0)

    # get predict from model
    yhat_class = model.predict(samples)
    yhat_prob = model.predict_proba(samples)

    # get label name
    class_index = yhat_class[0]
    predict_names = out_encoder.inverse_transform(yhat_class)[0]

    # get probability
    class_probability = round(yhat_prob[0,class_index] * 100)
    print('Predict image has person: ', predict_names, ' with confidence:', class_probability)

error: ignored